# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 [Online.](https://arxiv.org/abs/1611.06455 "Wang et al. (2017)")

## Data
Additional datasets are available at http://www.timeseriesclassification.com/. Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

In [5]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Train MLP
Expected running time for various datasets, on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset | Training time
:-------| :--------
Adiac   | 18 minutes


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os
import time

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

fdir = '../data'  
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'GunPoint', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['GunPoint']
for each in flist:
    fname = each
    print('Running dataset', fname)
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = int(min(x_train.shape[0]/10, 16))
    print('Batch size', batch_size)
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8)    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    end = time.time()
    print('Training complete on', fname)
    print('Training time (in seconds)', end-start)
    #Print the testing results which has the lowest training loss.
    print('Test result with the lowest training loss. Loss and accuracy are -')
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

 



Running dataset GunPoint
Batch size 5
Train on 50 samples, validate on 150 samples
Epoch 1/5000
50/50 [==============================] - 1s 14ms/step - loss: 0.7468 - acc: 0.5200 - val_loss: 0.5750 - val_acc: 0.6867
Epoch 2/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.5443 - acc: 0.7400 - val_loss: 0.5099 - val_acc: 0.7200
Epoch 3/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.3944 - acc: 0.8400 - val_loss: 0.4512 - val_acc: 0.7067
Epoch 4/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4657 - acc: 0.7600 - val_loss: 0.4062 - val_acc: 0.7933
Epoch 5/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.3807 - acc: 0.8200 - val_loss: 0.3774 - val_acc: 0.8067
Epoch 6/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.2820 - acc: 0.8200 - val_loss: 0.3921 - val_acc: 0.7800
Epoch 7/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.2806 - acc: 0.8800 - val_loss: 0.4413 - val_acc:

50/50 [==============================] - 0s 3ms/step - loss: 0.0445 - acc: 1.0000 - val_loss: 0.2777 - val_acc: 0.8933
Epoch 63/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0942 - acc: 0.9400 - val_loss: 0.2259 - val_acc: 0.9133
Epoch 64/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0271 - acc: 1.0000 - val_loss: 0.3681 - val_acc: 0.8667
Epoch 65/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0663 - acc: 0.9800 - val_loss: 0.2015 - val_acc: 0.9067
Epoch 66/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0381 - acc: 1.0000 - val_loss: 0.1916 - val_acc: 0.9200
Epoch 67/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.1640 - acc: 0.9200 - val_loss: 0.5353 - val_acc: 0.8067
Epoch 68/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0513 - acc: 1.0000 - val_loss: 0.1606 - val_acc: 0.9333
Epoch 69/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0619 -

50/50 [==============================] - 0s 3ms/step - loss: 0.0081 - acc: 1.0000 - val_loss: 0.3439 - val_acc: 0.9267
Epoch 124/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.3536e-04 - acc: 1.0000 - val_loss: 0.3094 - val_acc: 0.9400
Epoch 125/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.3152 - val_acc: 0.9400
Epoch 126/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.3461 - val_acc: 0.9267
Epoch 127/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.3191 - val_acc: 0.9333
Epoch 128/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.3860e-04 - acc: 1.0000 - val_loss: 0.3169 - val_acc: 0.9400
Epoch 129/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0068 - acc: 1.0000 - val_loss: 0.3737 - val_acc: 0.9133
Epoch 130/5000
50/50 [==============================] - 0s 3ms/step -

Epoch 183/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.7914e-05 - acc: 1.0000 - val_loss: 0.4117 - val_acc: 0.9333
Epoch 184/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.2000e-04 - acc: 1.0000 - val_loss: 0.4563 - val_acc: 0.9267
Epoch 185/5000
50/50 [==============================] - 0s 3ms/step - loss: 9.1940e-05 - acc: 1.0000 - val_loss: 0.4301 - val_acc: 0.9333
Epoch 186/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0077 - acc: 1.0000 - val_loss: 0.4220 - val_acc: 0.9400
Epoch 187/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.5009e-05 - acc: 1.0000 - val_loss: 0.4127 - val_acc: 0.9400
Epoch 188/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.9061e-05 - acc: 1.0000 - val_loss: 0.4106 - val_acc: 0.9400
Epoch 189/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.8015e-05 - acc: 1.0000 - val_loss: 0.4020 - val_acc: 0.9400
Epoch 190/5000
50/50 [================

50/50 [==============================] - 0s 3ms/step - loss: 2.8194e-05 - acc: 1.0000 - val_loss: 0.3897 - val_acc: 0.9333
Epoch 243/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.8252e-05 - acc: 1.0000 - val_loss: 0.3909 - val_acc: 0.9333
Epoch 244/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.1540e-05 - acc: 1.0000 - val_loss: 0.3917 - val_acc: 0.9333
Epoch 245/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.6715e-05 - acc: 1.0000 - val_loss: 0.3940 - val_acc: 0.9333
Epoch 246/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.8721e-05 - acc: 1.0000 - val_loss: 0.3896 - val_acc: 0.9400
Epoch 247/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.3201e-05 - acc: 1.0000 - val_loss: 0.3923 - val_acc: 0.9400
Epoch 248/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.7814e-05 - acc: 1.0000 - val_loss: 0.4260 - val_acc: 0.9267
Epoch 249/5000
50/50 [===========================

50/50 [==============================] - 0s 3ms/step - loss: 3.4205e-04 - acc: 1.0000 - val_loss: 0.5316 - val_acc: 0.9067
Epoch 302/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.0920e-05 - acc: 1.0000 - val_loss: 0.4651 - val_acc: 0.9333
Epoch 303/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.8505e-06 - acc: 1.0000 - val_loss: 0.4629 - val_acc: 0.9333
Epoch 304/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.7151e-06 - acc: 1.0000 - val_loss: 0.4642 - val_acc: 0.9333
Epoch 305/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.4295e-06 - acc: 1.0000 - val_loss: 0.4647 - val_acc: 0.9333
Epoch 306/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.2069e-06 - acc: 1.0000 - val_loss: 0.4598 - val_acc: 0.9333
Epoch 307/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.3393e-05 - acc: 1.0000 - val_loss: 0.4702 - val_acc: 0.9333
Epoch 308/5000
50/50 [===========================

50/50 [==============================] - 0s 3ms/step - loss: 8.0833e-06 - acc: 1.0000 - val_loss: 0.4545 - val_acc: 0.9400
Epoch 361/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.8883e-06 - acc: 1.0000 - val_loss: 0.4546 - val_acc: 0.9400
Epoch 362/5000
50/50 [==============================] - 0s 3ms/step - loss: 9.8728e-06 - acc: 1.0000 - val_loss: 0.4558 - val_acc: 0.9333
Epoch 363/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.0735e-06 - acc: 1.0000 - val_loss: 0.4563 - val_acc: 0.9333
Epoch 364/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.9525e-05 - acc: 1.0000 - val_loss: 0.4930 - val_acc: 0.9200
Epoch 365/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.0157e-05 - acc: 1.0000 - val_loss: 0.4629 - val_acc: 0.9333
Epoch 366/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.7169e-06 - acc: 1.0000 - val_loss: 0.4633 - val_acc: 0.9333
Epoch 367/5000
50/50 [===========================

# Recorded results
## Adiac

|Run |Loss |Accuracy | Comment
|:---|:--- |:---    |:----------
|1   |0.005222544357037315  |0.6930946294608933 |Adadelta - default values
|2   |0.0018498263113997382 |0.7109974427601261 |
|3   |0.001620212956988372  |0.7212276216358176 |Adadelta parameters match Wang (2017)